In [1]:
import pandas as pd
import numpy as np
# Assuming df is your DataFrame
df = pd.read_csv('/Users/praveenzirali/active_projects/NHIS 2019/adult19.csv')

# Age groups - crc
df['agecrc'] = None
df.loc[df['AGEP_A'].between(50, 64, inclusive='left'), 'agecrc'] = 1  # 50 <= agep_a < 65
df.loc[df['AGEP_A'].between(65, 75, inclusive='right'), 'agecrc'] = 2  # 65 <= agep_a <= 75

# CRC 50-75
df['ageadjc'] = None
df.loc[df['AGEP_A'].between(50, 64, inclusive='left'), 'ageadjc'] = 1
df.loc[df['AGEP_A'].between(65, 75, inclusive='right'), 'ageadjc'] = 2

# Handling specific values in canev_a
df.loc[df['CANEV_A'].isin([7, 8, 9]), 'CANEV_A'] = None

# CRC cancer
df['crccan'] = None
df.loc[(df['CANEV_A'] == 1) & (df['COLRCCAN_A'] == 1), 'crccan'] = 1
df.loc[(df['CANEV_A'] == 2) | (df['COLRCCAN_A'] == 2), 'crccan'] = 2

# FIT/FOBT in past 1 and 3 years
df['hfob1yr'] = None
df['hfob3yr'] = None
df.loc[df['FITHEV_A'] == 2, ['hfob1yr', 'hfob3yr']] = 2
df.loc[(df['FITHEV_A'] == 1) & (df['FITHWHEN_A'] == 1), 'hfob1yr'] = 1
df.loc[(df['FITHEV_A'] == 1) & (df['FITHWHEN_A'].between(2, 6)), 'hfob1yr'] = 2
df.loc[(df['FITHEV_A'] == 1) & (df['FITHWHEN_A'].between(1, 3)), 'hfob3yr'] = 1
df.loc[(df['FITHEV_A'] == 1) & (df['FITHWHEN_A'].between(4, 6)), 'hfob3yr'] = 2

# Sigmoidoscopy in past 5 years
df['sig5yr'] = None
df.loc[df['COLORECTEV_A'] == 2, 'sig5yr'] = 2
condition_sig = (df['COLORECTEV_A'] == 1) & (df['COLORECTYP_A'].isin([2, 3]))
df.loc[condition_sig & (df['SIGWHEN_A'].isin(range(1, 5))), 'sig5yr'] = 1
df.loc[condition_sig & (df['SIGWHEN_A'].isin([5, 6])), 'sig5yr'] = 2
df.loc[(df['COLORECTEV_A'] == 1) & (df['COLORECTYP_A'] == 1), 'sig5yr'] = 2
df.loc[df['sig5yr'].isnull() & (df['COLSIGWHEN_A'].isin([5, 6])), 'sig5yr'] = 2

# Colonoscopy in past 10 years
df['col10yr'] = None
df.loc[df['COLORECTEV_A'] == 2, 'col10yr'] = 2
condition_col = (df['COLORECTEV_A'] == 1) & (df['COLORECTYP_A'].isin([1, 3]))
df.loc[condition_col & (df['COLWHEN_A'].isin(range(1, 6))), 'col10yr'] = 1
df.loc[condition_col & (df['COLWHEN_A'].isin([6])), 'col10yr'] = 2
df.loc[(df['COLORECTEV_A'] == 1) & (df['COLORECTYP_A'] == 2), 'col10yr'] = 2
df.loc[df['col10yr'].isnull() & (df['COLSIGWHEN_A'] == 6), 'col10yr'] = 2

# CT colonography 5 years
df['ct5yr'] = None
df.loc[df['CTCOLEV_A'] == 2, 'ct5yr'] = 2
df.loc[(df['CTCOLEV_A'] == 1) & (df['CTCOLWHEN_A'].between(1, 4)), 'ct5yr'] = 1
df.loc[(df['CTCOLEV_A'] == 1) & (df['CTCOLWHEN_A'].between(5, 6)), 'ct5yr'] = 2

# Cologuard within 3 years
df['colgrd3yr'] = None
df.loc[df['COLOGUARD_A'] == 2, 'colgrd3yr'] = 2
condition_fitcolg = (df['COLOGUARD_A'] == 1) & (df['FITCOLG_A'] == 1)
df.loc[condition_fitcolg & (df['FITHWHEN_A'].isin([1, 2, 3])), 'colgrd3yr'] = 1
df.loc[condition_fitcolg & (df['FITHWHEN_A'].isin([4, 5, 6])), 'colgrd3yr'] = 2

# Define up to date with CRC exam
df['anycre'] = None
# Any of the exams within the recommended period
df.loc[(df['hfob1yr'] == 1) | (df['col10yr'] == 1) | (df['ct5yr'] == 1) | (df['sig5yr'] == 1) | (df['colgrd3yr'] == 1), 'anycre'] = 1
# None of the exams within the recommended period and all are too old
df.loc[(df['hfob1yr'] == 2) & (df['col10yr'] == 2) & (df['ct5yr'] == 2) & (df['sig5yr'] == 2) & (df['colgrd3yr'] == 2), 'anycre'] = 2
# Fill in cases where the only thing missing is the type of endoscopy
df.loc[df['anycre'].isnull() & (df['COLSIGWHEN_A'].between(1, 4)), 'anycre'] = 1
df.loc[df['anycre'].isnull() & (df['hfob1yr'] == 2) & (df['ct5yr'] == 2) & (df['colgrd3yr'] == 2) & (df['COLSIGWHEN_A'] == 6), 'anycre'] = 2

# FOBT/FIT or Cologuard composite
df['hfob_colgrd'] = None
df.loc[(df['hfob1yr'] == 1) | (df['colgrd3yr'] == 1), 'hfob_colgrd'] = 1
df.loc[(df['hfob1yr'] == 2) & (df['colgrd3yr'] == 2), 'hfob_colgrd'] = 2

# Assuming the formatting lines in SAS translate to column type definitions or categorical variable assignments in pandas
# For demonstration, a simple example of converting a column to a categorical type with ordered categories:
# df['educ'] = pd.Categorical(df['educ'], categories=[1, 2, 3, 4], ordered=True)

# Sorting the DataFrame (equivalent to proc sort; by pstrat ppsu;)
df.sort_values(by=['PSTRAT', 'PPSU'], inplace=True)

# Frequency analysis (equivalent to proc freq;)
# For columns starting with 'ageadj', 'agecrc', and 'sex_a'
ageadj_cols = [col for col in df.columns if col.startswith('ageadj') or col == 'agecrc' or col == 'SEX_A']
for col in ageadj_cols:
    print(df[col].value_counts(dropna=False))

# Analysis for Colorectal tests among recommended population
recommended_population = df[(df['AGEP_A'].between(40, 85)) & (df['crccan'] == 2)]
colorectal_tests = ['hfob1yr', 'sig5yr', 'col10yr', 'ct5yr', 'hfob3yr', 'colgrd3yr', 'anycre']
for test in colorectal_tests:
    print(recommended_population[test].value_counts(dropna=False))

SEX_A
2    17261
1    14733
7        3
Name: count, dtype: int64
agecrc
None    19215
1        7746
2        5036
Name: count, dtype: int64
ageadjc
None    19215
1        7746
2        5036
Name: count, dtype: int64
hfob1yr
None    18476
2        2323
1        1187
Name: count, dtype: int64
sig5yr
2       20994
1         681
None      311
Name: count, dtype: int64
col10yr
1       11807
2        9809
None      370
Name: count, dtype: int64
ct5yr
None    18459
2        3295
1         232
Name: count, dtype: int64
hfob3yr
None    18476
1        1871
2        1639
Name: count, dtype: int64
colgrd3yr
None    19519
2        2055
1         412
Name: count, dtype: int64
anycre
1       12615
None     9045
2         326
Name: count, dtype: int64


In [2]:
# Ensure your DataFrame has an age column and it's correctly named.
# For this example, I'm assuming the age column is named 'AGE' and it represents the respondent's age.
 
# Total number of respondents
total_respondents = len(df)
 
# Total number of respondents aged 50 to 75
respondents_40_85 = len(df[(df['AGEP_A'] >= 50) & (df['AGEP_A'] <= 75)])
total_respondents_40_85 = len(df[(df['AGEP_A'] >= 50) & (df['AGEP_A'] <= 75) & (df['crccan'] == 2)])
 
total_respondents_50_75 = len(df[
    (df['AGEP_A'] >= 50) & 
    (df['AGEP_A'] <= 75) & 
    (df['crccan'] == 2) & 
    (df['anycre'].isin([1,2]))
])
 
cancer_respondents = len(df[(df['AGEP_A'] >= 50) & (df['AGEP_A'] <= 75) &  (df['crccan'] != 2)])
# Number of compliant respondents aged 50 to 75
# Assuming 'compliance' is determined by the 'anycre' column being 1
compliant_respondents_50_75 = len(df[(df['AGEP_A'] >= 50) & (df['AGEP_A'] <= 75)  & (df['anycre'] == 1) & (df['crccan'] == 2)])
 
# Printing the results
print("Total Number of Respondents in NHIS data:", total_respondents)
print("Total number of respondents for the age 50 to 75:", respondents_40_85)
print("Total Number of eligible Respondents:", total_respondents_40_85)
print("Total Number of Respondents aged 50 to 75:", total_respondents_50_75)
print("Number of Compliant Respondents aged 50 to 75:", compliant_respondents_50_75)
print("Total Cancer Respondents:", cancer_respondents)

Total Number of Respondents in NHIS data: 31997
Total number of respondents for the age 50 to 75: 13989
Total Number of eligible Respondents: 13851
Total Number of Respondents aged 50 to 75: 9721
Number of Compliant Respondents aged 50 to 75: 9530
Total Cancer Respondents: 138
